In [22]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import seaborn as sns
import nibabel as nib
from os import listdir
from os.path import isfile, join
from Helpers import save_nifti

from scipy import signal
import lib

In [24]:
# topup exception
# analysis_id = 26 < 36
# => phase-encoding dimension
PE_dim = 'AP'

# current path
cur_path = os.getcwd()
# file name and path
file = "data"
file_name = cur_path + "/" + file + "/dfMRI_denoised_unringed.nii.gz"
# loading nifti object/header
dfMRI_denoised_unringed_lin = nib.load(file_name)
header_lin  = dfMRI_denoised_unringed_lin.header
affine_lin = dfMRI_denoised_unringed_lin.affine
dfMRI_denoised_unringed_lin = dfMRI_denoised_unringed_lin.get_fdata()

file = "BB_003_3T_cmrr_mbep2d_diff_0pt2_1_task_18s_off_12s_on_20200710180104_8"
file_name = cur_path + "/" + file + "/BB_003_3T_cmrr_mbep2d_diff_0pt2_1_revPE_20200710180104_9.nii.gz"
# loading nifti object/header
raw_data_revPE = nib.load(file_name)
header_revPE  = raw_data_revPE.header
affine_revPE = raw_data_revPE.affine
raw_data_revPE = raw_data_revPE.get_fdata()

No_slice_lin = header_lin['dim'][3]
No_slice_revPE = header_revPE['dim'][3]

In [6]:
No_slice_lin, No_slice_revPE

(14, 16)

In [25]:
save_nifti(dfMRI_denoised_unringed_lin[...,np.arange(0, dfMRI_denoised_unringed_lin.shape[-1], 2)],
            f'{cur_path}/data/topup/dfmri_linPE_b0_vols.nii.gz', affine_lin, header_lin)

save_nifti(raw_data_revPE, f'{cur_path}/data/topup/dfmri_revPE_vols.nii.gz', affine_revPE, header_revPE) 

In [26]:
dfmri_linPE_b0_vols = nib.load(f'{cur_path}/data/topup/dfmri_linPE_b0_vols.nii.gz')
dfmri_linPE_b0_vols = dfmri_linPE_b0_vols.get_fdata()
assert np.array_equal(dfmri_linPE_b0_vols, 
                      dfMRI_denoised_unringed_lin[...,np.arange(0, dfMRI_denoised_unringed_lin.shape[-1], 2)]), "The linPE_b0_vols is wrong"

# Apply -Tmean on linPE
linPE_b0_one_vol = np.mean(dfmri_linPE_b0_vols, axis=3)

save_nifti(linPE_b0_one_vol, f'{cur_path}/data/topup/dfmri_linPE_b0_one_vol.nii.gz', affine_lin, header_lin)

# Apply -Tmean on revPE
revPE_one_vol = np.mean(raw_data_revPE, axis=3)
save_nifti(revPE_one_vol, f'{cur_path}/data/topup/dfmri_revPE_one_vol.nii.gz', affine_revPE, header_revPE)


## Check that -Tmean is the mean according to axis 3 (time/b values)


In [10]:
import os

for PE in ['linPE_b0', 'revPE']:
    os.system('fslmaths '
            f'{cur_path}/data/topup/dfmri_{PE}_vols.nii.gz' 
            f' -Tmean {cur_path}/data/topup/fsl_tman_test.nii.gz')

    test_fsl_tmean = nib.load(f'{cur_path}/data/topup/fsl_tman_test.nii.gz')
    test_fsl_tmean = test_fsl_tmean.get_fdata()

    test_numpy = nib.load(cur_path + f'/data/topup/dfmri_{PE}_one_vol.nii.gz')
    test_numpy = test_numpy.get_fdata()
    assert np.array_equal(test_fsl_tmean, test_numpy), f"There is a difference between fslmaths -Tmean and np.mean({PE}, axis=3)"

    os.remove(f'{cur_path}/data/topup/fsl_tman_test.nii.gz') 

    del test_fsl_tmean, test_numpy

AssertionError: There is a difference between fslmaths -Tmean and np.mean(linPE_b0, axis=3)

## Check that fslroi is equivalent as image cropping

In [32]:
linPE_b0_one_vol = nib.load(f'{cur_path}/data/topup/dfmri_linPE_b0_one_vol.nii.gz')
linPE_b0_one_vol = linPE_b0_one_vol.get_fdata()

revPE_one_vol = nib.load(f'{cur_path}/data/topup/dfmri_revPE_one_vol.nii.gz')
revPE_one_vol = revPE_one_vol.get_fdata()

# If the number of z slice differs between linPE and revPE, 
# crop the larger image
if No_slice_lin < No_slice_revPE :
    start_idx = int((No_slice_revPE-No_slice_lin)/2)
    stop_idx = (int((No_slice_revPE-No_slice_lin)/2)+No_slice_lin)
    z_size = No_slice_lin
    revPE_one_vol = revPE_one_vol[:,:, start_idx:stop_idx] 
elif No_slice_lin > No_slice_revPE :
    start_idx = int((No_slice_lin-No_slice_revPE)/2)
    stop_idx = (int((No_slice_lin-No_slice_revPE)/2)+No_slice_revPE)
    z_size = No_slice_revPE
    linPE_b0_one_vol = linPE_b0_one_vol[:,:, start_idx:stop_idx] 
else :
    print("linPE and revPE have the same number of z slices")

save_nifti(linPE_b0_one_vol, f'{cur_path}/data/topup/dfmri_linPE_b0_one_vol_cropped.nii.gz', 
          affine_lin, header_lin)

save_nifti(revPE_one_vol, f'{cur_path}/data/topup/dfmri_revPE_one_vol_cropped.nii.gz', 
          affine_revPE, header_revPE)

In [33]:
if No_slice_lin < No_slice_revPE :
    to_crop = 'revPE'
    
elif No_slice_lin > No_slice_revPE :
    to_crop = 'linPE_b0'

os.system('fslroi '
          f'{cur_path}/data/topup/dfmri_{to_crop}_one_vol.nii.gz ' 
          f'{cur_path}/data/topup/dfmri_{to_crop}_fslroi.nii.gz ' 
          '0 -1 0 -1 '
          f'{int(abs(No_slice_revPE-No_slice_lin)/2)} '
          f'{z_size} 0 -1')

test_fslroi = nib.load(f'{cur_path}/data/topup/dfmri_{to_crop}_fslroi.nii.gz')
test_fslroi = test_fslroi.get_fdata()
test_numpy = nib.load(f'{cur_path}/data/topup/dfmri_{to_crop}_one_vol_cropped.nii.gz')
test_numpy = test_numpy.get_fdata()

assert np.array_equal(test_fslroi, test_numpy), f"The cropped version of {to_crop} is different from the fslroi one"

del test_fslroi, test_numpy


In [29]:
dfmri_linPE_b0_vols.shape

(116, 116, 14, 300)

## Check that fslmerge -t is actually merging 2 volumes according to the time (axis 3)

In [35]:
linPE_revPE = np.concatenate((linPE_b0_one_vol[...,np.newaxis], revPE_one_vol[...,np.newaxis]), axis=3)
save_nifti(linPE_revPE, f'{cur_path}/data/topup/linPE_revPE.nii.gz', affine_lin)

os.system('fslmerge -t '
         f'{cur_path}/data/topup/test_fslmerge.nii.gz '
         f'{cur_path}/data/topup/dfmri_linPE_b0_one_vol.nii.gz '
         f'{cur_path}/data/topup/dfmri_{to_crop}_one_vol_cropped.nii.gz ')

test_fslmerge = nib.load(f'{cur_path}/data/topup/test_fslmerge.nii.gz')
test_fslmerge = test_fslmerge.get_fdata()
assert np.array_equal(test_fslmerge, linPE_revPE), "Fslmerge and np.concatenate lead to different linPE_revPE array"

del test_fslmerge

AssertionError: Fslmerge and np.concatenate lead to different linPE_revPE array

## Calculate deformation field topup with linPE_revPE volume

### 1. Pad the image on z-axis

In [36]:
def image_padding(input_img: np.ndarray, axis_padding: int=2) -> np.ndarray:
    """
    Function that padds the image according to an axis
    Args:
        input_img (np.ndarray)
        axis_padding (int): axis along which to do the padding. Either 0, 1 or 2
    Returns:
        The padded volume (np.ndarray)
    """
    match axis_padding:
        case 0:
            first_plane = input_img[0,...]
            first_plane = first_plane[np.newaxis,...]
            last_plane = input_img[-1,...]
            last_plane = last_plane[np.newaxis,...]
        case 1:
            first_plane = input_img[:,0,:,:]
            first_plane = first_plane[:,np.newaxis,:,:]
            last_plane = input_img[:,-1,:,:]
            last_plane = last_plane[:,np.newaxis,:,:]
        case 2:
            first_plane = input_img[:,:,0,:]
            first_plane = first_plane[:,:,np.newaxis,:]
            last_plane = input_img[:,:,-1,:]
            last_plane = last_plane[:,:,np.newaxis,:]
        case _:
            print("The padding can only be performed on axis = 0, 1 or 2")
        
    return np.concatenate((first_plane, input_img, last_plane), axis=axis_padding)

## Check that fslroi is equivalent as image cropping

In [37]:
linPE_revPE_zpadded = image_padding(linPE_revPE, axis_padding=2)

save_nifti(linPE_revPE_zpadded, f'{cur_path}/data/topup/dfmri_linPE_revPE_padded.nii.gz', affine_lin)

### 2. Calculate the deformation field

In [38]:
os.system('topup --imain='
          f'{cur_path}/data/topup/dfmri_linPE_revPE_padded.nii.gz '
          f'--datain={cur_path}/acqp_{PE_dim}.txt '
          f'--out={cur_path}/data/topup/topup_res '
          f'--iout={cur_path}/data/topup/dfmri_linPE_revPE_tu.nii.gz '
          f'--config={cur_path}/topup.cnf -v '
          f'--fout={cur_path}/data/topup/fieldval.nii.gz')    

SSD = 918.989	n = 19488	Reg = 0	Cost = 918.989
SSD = 316.81	n = 18053	Reg = 33.7919	Cost = 350.602
SSD = 145.555	n = 18625	Reg = 50.3623	Cost = 195.917
SSD = 78.0492	n = 19488	Reg = 46.9801	Cost = 125.029
SSD = 54.5178	n = 19488	Reg = 44.1318	Cost = 98.6496
SSD = 45.504	n = 19488	Reg = 42.8688	Cost = 88.3728
***Going to next resolution level***
SSD = 84.254	n = 19488	Reg = 21.2105	Cost = 105.464
SSD = 47.2753	n = 19488	Reg = 16.5186	Cost = 63.7938
SSD = 38.5657	n = 19488	Reg = 17.5082	Cost = 56.0739
SSD = 36.7188	n = 19488	Reg = 18.1272	Cost = 54.846
SSD = 36.2498	n = 19488	Reg = 18.3231	Cost = 54.5729
SSD = 36.1029	n = 19488	Reg = 18.3891	Cost = 54.4919
***Going to next resolution level***
SSD = 83.715	n = 19488	Reg = 4.28162	Cost = 87.9967
SSD = 67.4039	n = 19488	Reg = 5.85578	Cost = 73.2596
SSD = 64.4675	n = 19488	Reg = 6.898	Cost = 71.3655
SSD = 63.7489	n = 19488	Reg = 7.27576	Cost = 71.0247
SSD = 63.5059	n = 19488	Reg = 7.43057	Cost = 70.9365
SSD = 63.383	n = 19488	Reg = 7.51468	C

0

### 3. Pad the data on which you want to apply topup on the z-axis as well

In [39]:
file_name = f"{cur_path}/data/dfMRI_denoised_unringed.nii.gz"
# loading nifti object
dfmri_degibbs = nib.load(file_name)
dfmri_degibbs = dfmri_degibbs.get_fdata()

dfmri_degibbs_zpadded = image_padding(dfmri_degibbs, axis_padding=2)

save_nifti(dfmri_degibbs_zpadded, f'{cur_path}/data/topup/dfmri_degibbs_padded.nii.gz', affine_lin, header_lin)

### 4. Apply topup

In [40]:
os.system('applytopup -i '
          f'{cur_path}/data/topup/dfmri_degibbs_padded.nii.gz '
          '--inindex=1 '
          f'--datain={cur_path}/acqp_{PE_dim}.txt '
          f'--topup={cur_path}/data/topup/topup_res '
          f'--out={cur_path}/data/topup/dfmri_deggibs_padded_uw '
          '--method=jac')

0

### 5. Remove z-padding

In [41]:
dfmri_deggibs_padded_uw = nib.load(f'{cur_path}/data/topup/dfmri_deggibs_padded_uw.nii.gz')
dfmri_deggibs_padded_uw = dfmri_deggibs_padded_uw.get_fdata()
dfmri_deggibs_sdc = dfmri_deggibs_padded_uw[:,:,1:z_size+1,:]

save_nifti(dfmri_deggibs_sdc, f'{cur_path}/data/topup/dfmri_degibbs_sdc.nii.gz', affine_lin, header_lin)